# Finantier Assessment

## Data Preprocessing

In [1]:
import pandas as pd

pd.set_option('max_columns', None)

In [4]:
# configs
fp_data = '../project/data/finantier_data_technical_test_dataset.csv'

In [5]:
df = pd.read_csv(fp_data)

In [6]:
df.columns

Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Default',
       'ErrorIndicator'],
      dtype='object')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7048 entries, 0 to 7047
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7044 non-null   object 
 1   gender            7044 non-null   object 
 2   SeniorCitizen     7043 non-null   float64
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   float64
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [8]:
df[~df.ErrorIndicator.isnull()]

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Default,ErrorIndicator
0,4808-GHDJN,Female,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,error_error
2,5575-GNVDE,Male,0.0,No,No,34.0,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No,error_error
27,6322-HRPFA,Male,0.0,Yes,Yes,49.0,Yes,No,DSL,Yes,Yes,No,Yes,No,No,Month-to-month,No,Credit card (automatic),59.60,2970.3,No,error_error
83,0191-ZHSKZ,Male,1.0,No,No,30.0,Yes,No,DSL,Yes,Yes,No,No,Yes,Yes,Month-to-month,Yes,Electronic check,74.75,2111.3,No,error_error


In [17]:
df.OnlineBackup.value_counts(dropna=False)

No                     3088
Yes                    2429
No internet service    1526
NaN                       5
Name: OnlineBackup, dtype: int64

In [ ]:
def load_data(fp):
    pd.read_csv(fp)

def process_telecom_data(df):
    # remove error rows
    df = df[df.ErrorIndicator.isnull()]

    # label encode features
    mappings = {'0':0, 'a':1, 'b':2, 'c':3, 'd':4}
    df['StoreType'] = data.StoreType.map(mappings).fillna(0)
    df['Assortment'] = data.Assortment.map(mappings).fillna(0)
    df['StateHoliday'] = data.StateHoliday.map(mappings).fillna(0)

    
    
